In [11]:
import pandas as pd
import re
import glob
import os

os.getcwd()

'/Users/chance/Desktop/Project/세라잼/DCX'

# 크롤링 파일 이름명 예시

[프로젝트명]Crawling_채널이름채널숫자.csv

ex. [세라잼]Crawling_네이버카페0.csv

In [12]:
def preprocess_text(text):
    if isinstance(text, str):
        text = re.sub(r'\n', ' ', text) # 줄넘김 없앰
        text = re.sub(r'\s+', ' ', text) # 공백 두 개 이상은 공백 하나로
        text = re.sub(r'[^\w\s]', ' ', text) # 특수문자 제거
        text = re.sub(r'http[s]?://\S+', ' ', text) #re.sub(r'https?://\S+', '', text) url 제거
        text = re.sub(r'\d', ' ', text) # 숫자 제거

    return text

In [13]:
def main(project_name, channel, crawling_data_path, labeling_result_path,group_num, sample_frac=0.05):
    data = os.listdir(crawling_data_path)

    label_list = []
    for c in channel:
        crawling_file_list = glob.glob(f'{crawling_data_path}/*{c}*.csv')
        file_list = [pd.read_csv(file) for file in crawling_file_list]
        
        globals()[f"[{project_name}]Crawling_{c}"] = pd.concat(file_list, axis=0)
        globals()[f"[{project_name}]Crawling_{c}"]['review'] = globals()[f"[{project_name}]Crawling_{c}"]['review'].apply(preprocess_text)
        globals()[f"[{project_name}]Crawling_{c}"].to_csv(f'./data/[{project_name}]Crawling_{c}.csv', encoding='utf-8-sig', index=False)

        globals()[f"[{project_name}]Crawling_{c}_labeling"] = globals()[f"[{project_name}]Crawling_{c}"].sample(n=None, frac=sample_frac, replace=False, weights=None, random_state=42, axis=None, ignore_index=False)

        label_list.append(globals()[f"[{project_name}]Crawling_{c}_labeling"])
    
    label_merged = pd.concat(label_list, axis=0)
    label_merged['0 or 1'] = 0
    
    temp = len(label_merged['review'])//group_num
    for i in range(group_num):
        lebel_grouped = label_merged.iloc[temp*i:temp*(i+1)]
        lebel_grouped.to_csv(f'{labeling_result_path}/[{project_name}]Labeling_group{i}.csv', index=False)

In [14]:
project_name = '세라젬'
channel = ['musinsa', 'naver']
group_num = 4

crawling_data_path = './0.크롤링/result' #크롤링 파일 위치
labeling_result_path = './1.전처리_라벨링파일생성/result' #저장할 라벨링 파일 위치
sample_frac = 0.02

main(project_name, channel, crawling_data_path, labeling_result_path, group_num, sample_frac)